In [1]:
import girder_client
import os
import cv2
import json
from IPython.display import Image
import skimage.segmentation  ## from scikit-images
import numpy as np
import matplotlib.pyplot as plt
from cStringIO import StringIO
import numpy as np
from PIL import Image as PILImage
import PIL
import image_processing_aug2016 as ip
#from ScikitSegmentationHelper import ScikitSegmentationHelper
import numpy

In [2]:
gc = girder_client.GirderClient(apiUrl="http://digitalslidearchive.emory.edu:8080/api/v1")

In [3]:
TCGACohortList = gc.get('/tcga/cohort')
cohortData = {}
for x in TCGACohortList['data']:
    cohortData[x['lowerName']] = { '_id': x['_id'], 'lowerName': x['lowerName']}

In [4]:
gbmImageSet = gc.get('/tcga/cohort/%s/images?limit=10' % cohortData['gbm']['_id'])
sampleItemToGrab = gbmImageSet['data'][4]

imgWidth = 1024
curImageMacroUrl = '/item/%s/tiles/thumbnail?width=%s' % (sampleItemToGrab['_id'],imgWidth)
curImage = gc.get(curImageMacroUrl, jsonResp=False)

In [5]:
def uint64ToRGB(val):
    return numpy.dstack((
        val.astype(numpy.uint8),
        (val >> numpy.uint64(8)).astype(numpy.uint8),
        (val >> numpy.uint64(16)).astype(numpy.uint8)
    ))


def superpixels(image):
    superpixelLabels = _slic(image, numSegments=1000)
    superpixels = uint64ToRGB(superpixelLabels)
    return superpixels


def _slic(image, numSegments=None, segmentSize=None):
        compactness = 0.01  # make superpixels highly deformable
        maxIter = 10
        sigma = 2.0

        if numSegments and segmentSize:
            raise ValueError(
                'Only one of numSegments or segmentSize may be set.')
        elif numSegments:
            pass
        elif segmentSize:
            numSegments = (image.shape[0] * image.shape[1]) / (segmentSize ** 2)
        else:
            raise ValueError('One of numSegments or segmentSize must be set.')

        labelImage = skimage.segmentation.slic(
            image,
            n_segments=numSegments,
            compactness=compactness,
            max_iter=maxIter,
            sigma=sigma,
           enforce_connectivity=True,
           min_size_factor=0.5,
            slic_zero=True
        )
        return labelImage

In [6]:
RawImageToPIL =  PILImage.open(StringIO(curImage.content)) ### This is painful.. 
spxImage = superpixels(np.array(RawImageToPIL))

In [7]:
spxImage = PILImage.fromarray(spxImage)

spxImage.save('sampleSuperPixelImage.png')



In [10]:
### Now we need to generate a JSONized version of the boudnaries
import numpy as np, cv2, geojson
### I have to switch coordinate systems, which is why I need to multiple by -1
def contourToSVGString(contour):
        """
        convert an OpenCV contour to a geojson-compatible representation
        """
        t_string = []
        for pt in contour:
            px = np.round(pt[0] )
            py =  np.round(pt[1] )  ## Appears Y is inverted/negative in the cntours
            t_string.append((float(px), float(py)))
        return t_string

##@dview.parallel(block=False)  << This is if I want to use ipyparallel
def TileMap_to_SVG( tileImg,useRawFile=False,debug=False ):
    """
    This function takes an input image as a PNG File and produces a pointlist of the
    outer most contour for each label map, and produces a format I can read in as SVG for visualization"""
    # Load an color image in grayscale; currently not working with > 256 superpixels

    if useRawFile:
        img = cv2.imread(tileImg)
    else:
        img = tileImg ### I am not reading from the filesystem in this case, I'm passing the opened iflept
    
    img_ch0 = img[:,:,0]
    img_ch1 = img[:,:,1]
    img_ch2 = img[:,:,2]
    
    img_minor = img_ch2  ## Need to double check this
    img_major = img_ch1   ## Also check this

    img_gray = 256 * img_major.astype(np.int) + img_minor.astype(np.int) +1 ##Allows > 256 Channels
    ### Probably should add a 1 to get rid of the 0th channel.. this is a TO DO
    
    unique_labels = np.unique(img_gray)
    if debug: print len(unique_labels)
    
    all_cnts = []
    cntdict = {}
    return_data = []
    ## So a given label/contour can contain one or more features...
    
    ## Now iterate through each label, mask the input image with the label, and find appropriate contours
    for label in unique_labels:
        working_img = img_gray.copy()
        working_img[working_img != label] = 0
        ### this breaks when label == 0
        
        ## can pass cv2.RETR_CCCOMP or RETR_EXTERNAL
        _,contours, hierarchy = cv2.findContours(working_img.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) ## can also do CHAIN_APPROX_TC89_L1
        
        if len(contours) > 0:
            if debug: print "Found %d contours for label %s" % ( len(contours), label)
            for c in contours:    
                try:
                    outerpoly = ip.contourToSVGString( np.squeeze( c ) )            
                    all_cnts.append( { 'geometry': { 'type': 'polygon', 'coordinates': outerpoly }, 'properties' : { 'labelindex': str(label-1)    } } )
                except:
                    print "SOMETHING WRONG IN THIS IMAGE CONTOUR... 2 few points??",c
        else:
            if debug: print len(contours),"were found for label",label
    for c in all_cnts:
        return_data.append(c)

        
    superpixel_contours = return_data

#    return superpixel_contours
    
    if useRawFile:
        svg_output_file = tileImg+'.svg.json'   
        print svg_output_file
        if not os.path.isfile(svg_output_file):
            with  open(svg_output_file,'w') as outfile:
                json.dump(superpixel_contours, outfile)
        return len(return_data)
    else:
        return superpixel_contours

In [11]:
spxBoundaryFiles = TileMap_to_SVG('sampleSuperPixelImage.png',useRawFile=True)

sampleSuperPixelImage.png.svg.json
